In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns
from ModelTree import ModelTree
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA

In [3]:
# change according to where your dataset is stored locally
# data has a header already
# load the data
df = pd.read_csv(r"fps-in-video-games.csv")

/Users/andreibrasoveanu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,17,22,25,26,27,29,34,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data exploration
Since this step was done more extensively in Exercise 0, we will explore the data only briefly.

In [4]:
# let's see the dimensions of our dataset
df.shape

(425833, 45)

In [5]:
# let's check the types of our attributes
df.dtypes

CpuName                       object
CpuNumberOfCores               int64
CpuNumberOfThreads             int64
CpuBaseClock                   int64
CpuCacheL1                     int64
CpuCacheL2                     int64
CpuCacheL3                    object
CpuDieSize                    object
CpuFrequency                   int64
CpuMultiplier                float64
CpuMultiplierUnlocked          int64
CpuProcessSize                 int64
CpuTDP                         int64
CpuNumberOfTransistors        object
CpuTurboClock                  int64
GpuName                       object
GpuArchitecture               object
GpuBandwidth                  object
GpuBaseClock                   int64
GpuBoostClock                  int64
GpuBus nterface               object
GpuNumberOfComputeUnits       object
GpuDieSize                    object
GpuDirectX                    object
GpuNumberOfExecutionUnits     object
GpuFP32Performance            object
GpuMemoryBus                  object
G

In [6]:
#include all columns in the output
pd.options.display.max_columns = df.shape[1]
df.describe(include="all")

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuNumberOfTransistors,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuNumberOfComputeUnits,GpuDieSize,GpuDirectX,GpuNumberOfExecutionUnits,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
count,425833,425833.000000,425833.000000,425833.000000,425833.000000,425833.000000,425833,425833,425833.000000,425833.000000,425833.000000,425833.000000,425833.000000,425833,425833.000000,425833,425833,425833,425833.000000,425833.000000,425833,425833,425833,425833,425833,425833,425833,425833,425833,425833,425833.000000,425833.000000,425833.000000,425833.000000,425833.0,425833,425833.000000,425833.00000,425833,425833,425833,425833.000000,425833,425833,425833.000000
unique,358,NaN,NaN,NaN,NaN,NaN,29,37,NaN,NaN,NaN,NaN,NaN,29,NaN,446,28,163,NaN,NaN,17,27,120,8,6,565,25,31,9,10,NaN,NaN,NaN,NaN,19.0,123,NaN,NaN,121,7,30,NaN,4,2,NaN
top,'Intel Core i7-7700K',NaN,NaN,NaN,NaN,NaN,8,?,NaN,NaN,NaN,NaN,NaN,?,NaN,'NVIDIA GeForce GTX 1060 6 GB 9Gbps',Pascal,256300,NaN,NaN,'PCIe 3.0 x16',?,0.000314,12,?,4375000,256,8000,GDDR5,1.2,NaN,NaN,NaN,NaN,6.4,1280,NaN,NaN,7200,1.2.131,counterStrikeGlobalOffensive,NaN,max,userbenchmark,NaN
freq,33956,NaN,NaN,NaN,NaN,NaN,129274,202534,NaN,NaN,NaN,NaN,NaN,226039,NaN,58910,235245,62220,NaN,NaN,382955,357107,86497,391599,412935,33318,202462,146444,243797,252467,NaN,NaN,NaN,NaN,265709.0,35444,NaN,NaN,86497,327634,101271,NaN,210221,401209,NaN
mean,NaN,4.879042,7.893435,108.595719,348.152257,1989.470840,NaN,NaN,3495.853123,33.422252,0.568617,17.296017,81.365453,NaN,3998.218353,NaN,NaN,NaN,1283.728107,1443.065807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.580215,73316.627643,18.511745,48.479458,NaN,NaN,111.165349,166699.74474,NaN,NaN,NaN,1065.087957,NaN,NaN,138.545663
std,NaN,1.722288,3.838796,28.173990,168.379841,1805.837566,NaN,NaN,445.981960,6.455594,0.495270,7.029708,22.068487,NaN,490.780893,NaN,NaN,NaN,322.646809,299.773732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125928,38603.080486,7.090753,21.607163,NaN,NaN,55.468640,93522.36617,NaN,NaN,NaN,162.661232,NaN,NaN,87.818185
min,NaN,1.000000,1.000000,100.000000,64.000000,512.000000,NaN,NaN,1500.000000,8.000000,0.000000,7.000000,10.000000,NaN,1600.000000,NaN,NaN,NaN,100.000000,350.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,425.000000,7.000000,1.000000,NaN,NaN,2.000000,850.00000,NaN,NaN,NaN,720.000000,NaN,NaN,0.000000
25%,NaN,4.000000,4.000000,100.000000,256.000000,1024.000000,NaN,NaN,3300.000000,32.000000,0.000000,14.000000,65.000000,NaN,3700.000000,NaN,NaN,NaN,1050.000000,1178.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,40510.000000,14.000000,32.000000,NaN,NaN,80.000000,87840.00000,NaN,NaN,NaN,1080.000000,NaN,NaN,80.000000
50%,NaN,4.000000,8.000000,100.000000,256.000000,1024.000000,NaN,NaN,3500.000000,35.000000,1.000000,14.000000,88.000000,NaN,4000.000000,NaN,NaN,NaN,1410.000000,1582.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,82030.000000,16.000000,48.000000,NaN,NaN,120.000000,154400.00000,NaN,NaN,NaN,1080.000000,NaN,NaN,120.000000
75%,NaN,6.000000,12.000000,100.000000,384.000000,2048.000000,NaN,NaN,3700.000000,37.000000,1.000000,22.000000,95.000000,NaN,4350.000000,NaN,NaN,NaN,1506.000000,1709.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,107700.000000,16.000000,64.000000,NaN,NaN,144.000000,202000.00000,NaN,NaN,NaN,1080.000000,NaN,NaN,180.000000


In [7]:
# let's inspect the columns with the object type
# we can see that even though they have numeric values, missing values are marked with the '?' character, which makes them turn to object type
# we can see that we will need to deal with the missing values
df.loc[:,df.dtypes=="object"]

,CpuName,CpuCacheL3,CpuDieSize,CpuNumberOfTransistors,GpuName,GpuArchitecture,GpuBandwidth,GpuBus nterface,GpuNumberOfComputeUnits,GpuDieSize,GpuDirectX,GpuNumberOfExecutionUnits,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTransistors,GpuVulkan,GameName,GameSetting,Dataset
0,'Intel Core i7-920',8,0.000263,731,'AMD Radeon RX 480','GCN 4.0',256000,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,6.4,2304,5700,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
1,'Intel Core i7-920',8,0.000263,731,'AMD Radeon RX 480','GCN 4.0',256000,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,6.4,2304,5700,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
2,'Intel Core i5-4690',6,0.000177,1400,'NVIDIA GeForce GTX 1080 11Gbps',Pascal,352300,'PCIe 3.0 x16',?,0.000314,12,?,8873000,256,8000,GDDR5X,1.2,6.4,2560,7200,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
3,'Intel Core i5-4690',6,0.000177,1400,'NVIDIA GeForce GTX 1080 11Gbps',Pascal,352300,'PCIe 3.0 x16',?,0.000314,12,?,8873000,256,8000,GDDR5X,1.2,6.4,2560,7200,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
4,'Intel Core i5-4690',6,0.000177,1400,'AMD Radeon R7 260X','GCN 2.0',104000,'PCIe 3.0 x16',14,0.00016,12,?,1971000,128,2000,GDDR5,2,6.3,896,2080,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
5,'Intel Core i5-4690',6,0.000177,1400,'AMD Radeon R7 260X','GCN 2.0',104000,'PCIe 3.0 x16',14,0.00016,12,?,1971000,128,2000,GDDR5,2,6.3,896,2080,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
6,'Intel Core i5-4690',6,0.000177,1400,'AMD Radeon R9 280X','GCN 1.0',288000,'PCIe 3.0 x16',32,0.000352,12,?,4096000,384,3000,GDDR5,1.2,5.1,2048,4313,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
7,'Intel Core i5-4690',6,0.000177,1400,'AMD Radeon R9 280X','GCN 1.0',288000,'PCIe 3.0 x16',32,0.000352,12,?,4096000,384,3000,GDDR5,1.2,5.1,2048,4313,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
8,'Intel Core i5-4690',6,0.000177,1400,'AMD Radeon RX 480','GCN 4.0',256000,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,6.4,2304,5700,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark
9,'Intel Core i5-4690',6,0.000177,1400,'AMD Radeon RX 480','GCN 4.0',256000,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,6.4,2304,5700,1.2.131,counterStrikeGlobalOffensive,low,userbenchmark


In [8]:
# checking for traditional NA values returns null
df.isna().any().any()

False

In [9]:
# get all the rows containing missing values
# we can see that each row contains at least a missing value
df[(df == '?').any(axis=1)]

/Users/andreibrasoveanu/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuNumberOfTransistors,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuNumberOfComputeUnits,GpuDieSize,GpuDirectX,GpuNumberOfExecutionUnits,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,'Intel Core i7-920',4,8,133,256,1024,8,0.000263,2666,20.0,0,45,130,731,2933,'AMD Radeon RX 480','GCN 4.0',256000,1120,1266,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,4.6,40510,14,32,6.4,2304,144,182300,5700,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,70.0
1,'Intel Core i7-920',4,8,133,256,1024,8,0.000263,2666,20.0,0,45,130,731,2933,'AMD Radeon RX 480','GCN 4.0',256000,1120,1266,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,4.6,40510,14,32,6.4,2304,144,182300,5700,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,230.0
2,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'NVIDIA GeForce GTX 1080 11Gbps',Pascal,352300,1607,1733,'PCIe 3.0 x16',?,0.000314,12,?,8873000,256,8000,GDDR5X,1.2,4.6,110900,16,64,6.4,2560,160,277300,7200,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,250.0
3,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'NVIDIA GeForce GTX 1080 11Gbps',Pascal,352300,1607,1733,'PCIe 3.0 x16',?,0.000314,12,?,8873000,256,8000,GDDR5X,1.2,4.6,110900,16,64,6.4,2560,160,277300,7200,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,300.0
4,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'AMD Radeon R7 260X','GCN 2.0',104000,1100,1100,'PCIe 3.0 x16',14,0.00016,12,?,1971000,128,2000,GDDR5,2,4.6,17600,28,16,6.3,896,56,61600,2080,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,200.0
5,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'AMD Radeon R7 260X','GCN 2.0',104000,1100,1100,'PCIe 3.0 x16',14,0.00016,12,?,1971000,128,2000,GDDR5,2,4.6,17600,28,16,6.3,896,56,61600,2080,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,200.0
6,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'AMD Radeon R9 280X','GCN 1.0',288000,850,1000,'PCIe 3.0 x16',32,0.000352,12,?,4096000,384,3000,GDDR5,1.2,4.6,32000,28,32,5.1,2048,128,128000,4313,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,200.0
7,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'AMD Radeon R9 280X','GCN 1.0',288000,850,1000,'PCIe 3.0 x16',32,0.000352,12,?,4096000,384,3000,GDDR5,1.2,4.6,32000,28,32,5.1,2048,128,128000,4313,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,250.0
8,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'AMD Radeon RX 480','GCN 4.0',256000,1120,1266,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,4.6,40510,14,32,6.4,2304,144,182300,5700,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,300.0
9,'Intel Core i5-4690',4,4,100,256,1024,6,0.000177,3400,34.0,0,22,84,1400,3800,'AMD Radeon RX 480','GCN 4.0',256000,1120,1266,'PCIe 3.0 x16',36,0.000232,12,?,5834000,256,8000,GDDR5,2,4.6,40510,14,32,6.4,2304,144,182300,5700,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,300.0


In [10]:
# get percentage of missing values per feature:
df[(df == '?')].count()/df.shape[0]

/Users/andreibrasoveanu/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


CpuName                      0.000000
CpuNumberOfCores             0.000000
CpuNumberOfThreads           0.000000
CpuBaseClock                 0.000000
CpuCacheL1                   0.000000
CpuCacheL2                   0.000000
CpuCacheL3                   0.011871
CpuDieSize                   0.475618
CpuFrequency                 0.000000
CpuMultiplier                0.000000
CpuMultiplierUnlocked        0.000000
CpuProcessSize               0.000000
CpuTDP                       0.000000
CpuNumberOfTransistors       0.530816
CpuTurboClock                0.000000
GpuName                      0.000000
GpuArchitecture              0.000000
GpuBandwidth                 0.036261
GpuBaseClock                 0.000000
GpuBoostClock                0.000000
GpuBus nterface              0.000000
GpuNumberOfComputeUnits      0.838608
GpuDieSize                   0.026292
GpuDirectX                   0.000000
GpuNumberOfExecutionUnits    0.969711
GpuFP32Performance           0.000132
GpuMemoryBus

In [11]:
# we can see that 4 features have lots of missing values (53%, 47%, 83% and 96% respectively)
# we will therefore delete them because too much data is missing to perform a reliable form of imputation
# this will also help by reducing the dimensionality of our data
selected_columns=["CpuDieSize" , "CpuNumberOfTransistors", "GpuNumberOfExecutionUnits", "GpuNumberOfComputeUnits"]
df=df.drop(columns=selected_columns)

In [12]:
df.dtypes

CpuName                     object
CpuNumberOfCores             int64
CpuNumberOfThreads           int64
CpuBaseClock                 int64
CpuCacheL1                   int64
CpuCacheL2                   int64
CpuCacheL3                  object
CpuFrequency                 int64
CpuMultiplier              float64
CpuMultiplierUnlocked        int64
CpuProcessSize               int64
CpuTDP                       int64
CpuTurboClock                int64
GpuName                     object
GpuArchitecture             object
GpuBandwidth                object
GpuBaseClock                 int64
GpuBoostClock                int64
GpuBus nterface             object
GpuDieSize                  object
GpuDirectX                  object
GpuFP32Performance          object
GpuMemoryBus                object
GpuMemorySize               object
GpuMemoryType               object
GpuOpenCL                   object
GpuOpenGL                  float64
GpuPixelRate                 int64
GpuProcessSize      

In [13]:
#selected_columns=["CpuCacheL3", "GpuDieSize", "GpuFP32Performance", "GpuMemoryBus","GpuMemorySize","GpuMemoryType","GpuOpenCL","GpuShaderModel", "GpuNumberOfShadingUnits","GpuNumberOfTransistors","GpuVulkan"]
#remove missing values
df = df[df["CpuCacheL3"] != '?']
df = df[df["GpuDieSize"] != '?']
df = df[df["GpuFP32Performance"] != '?']
df = df[df["GpuMemoryBus"] != '?']
df = df[df["GpuMemorySize"] != '?']
df = df[df["GpuMemoryType"] != '?']
df = df[df["GpuOpenCL"] != '?']
df = df[df["GpuShaderModel"] != '?']
df = df[df["GpuNumberOfShadingUnits"] != '?']
df = df[df["GpuNumberOfTransistors"] != '?']
df = df[df["GpuVulkan"] != '?']

In [14]:
df.head()

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuMultiplierUnlocked,CpuProcessSize,CpuTDP,CpuTurboClock,GpuName,GpuArchitecture,GpuBandwidth,GpuBaseClock,GpuBoostClock,GpuBus nterface,GpuDieSize,GpuDirectX,GpuFP32Performance,GpuMemoryBus,GpuMemorySize,GpuMemoryType,GpuOpenCL,GpuOpenGL,GpuPixelRate,GpuProcessSize,GpuNumberOfROPs,GpuShaderModel,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,'Intel Core i7-920',4,8,133,256,1024,8,2666,20.0,0,45,130,2933,'AMD Radeon RX 480','GCN 4.0',256000,1120,1266,'PCIe 3.0 x16',0.000232,12,5834000,256,8000,GDDR5,2,4.6,40510,14,32,6.4,2304,144,182300,5700,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,70.0
1,'Intel Core i7-920',4,8,133,256,1024,8,2666,20.0,0,45,130,2933,'AMD Radeon RX 480','GCN 4.0',256000,1120,1266,'PCIe 3.0 x16',0.000232,12,5834000,256,8000,GDDR5,2,4.6,40510,14,32,6.4,2304,144,182300,5700,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,230.0
2,'Intel Core i5-4690',4,4,100,256,1024,6,3400,34.0,0,22,84,3800,'NVIDIA GeForce GTX 1080 11Gbps',Pascal,352300,1607,1733,'PCIe 3.0 x16',0.000314,12,8873000,256,8000,GDDR5X,1.2,4.6,110900,16,64,6.4,2560,160,277300,7200,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,250.0
3,'Intel Core i5-4690',4,4,100,256,1024,6,3400,34.0,0,22,84,3800,'NVIDIA GeForce GTX 1080 11Gbps',Pascal,352300,1607,1733,'PCIe 3.0 x16',0.000314,12,8873000,256,8000,GDDR5X,1.2,4.6,110900,16,64,6.4,2560,160,277300,7200,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,300.0
4,'Intel Core i5-4690',4,4,100,256,1024,6,3400,34.0,0,22,84,3800,'AMD Radeon R7 260X','GCN 2.0',104000,1100,1100,'PCIe 3.0 x16',0.00016,12,1971000,128,2000,GDDR5,2,4.6,17600,28,16,6.3,896,56,61600,2080,1.2.131,counterStrikeGlobalOffensive,1080,low,userbenchmark,200.0


In [15]:
# check if the data types are correct
df.dtypes

CpuName                     object
CpuNumberOfCores             int64
CpuNumberOfThreads           int64
CpuBaseClock                 int64
CpuCacheL1                   int64
CpuCacheL2                   int64
CpuCacheL3                  object
CpuFrequency                 int64
CpuMultiplier              float64
CpuMultiplierUnlocked        int64
CpuProcessSize               int64
CpuTDP                       int64
CpuTurboClock                int64
GpuName                     object
GpuArchitecture             object
GpuBandwidth                object
GpuBaseClock                 int64
GpuBoostClock                int64
GpuBus nterface             object
GpuDieSize                  object
GpuDirectX                  object
GpuFP32Performance          object
GpuMemoryBus                object
GpuMemorySize               object
GpuMemoryType               object
GpuOpenCL                   object
GpuOpenGL                  float64
GpuPixelRate                 int64
GpuProcessSize      

In [16]:
# convert features that contained missing values to float type
features_to_convert=["CpuCacheL3","GpuBandwidth","GpuDieSize","GpuFP32Performance","GpuMemoryBus","GpuMemorySize","GpuOpenCL","GpuShaderModel","GpuNumberOfShadingUnits","GpuNumberOfTransistors"]
for i in features_to_convert:
    df[i]=pd.to_numeric(df[i])
    #print (df[i].unique())
#convert game resolution and GpuDirectX to object type, since they represents categorical data
df["GameResolution"]=df["GameResolution"].astype(object)
df["GpuDirectX"]=df["GpuDirectX"].astype(object)

In [17]:
# we can see that we have 396028/425833=93% of our original data, so still lots of data
df.shape

(396028, 41)

In [18]:
# check again that the data types are now alright
df.dtypes

CpuName                     object
CpuNumberOfCores             int64
CpuNumberOfThreads           int64
CpuBaseClock                 int64
CpuCacheL1                   int64
CpuCacheL2                   int64
CpuCacheL3                   int64
CpuFrequency                 int64
CpuMultiplier              float64
CpuMultiplierUnlocked        int64
CpuProcessSize               int64
CpuTDP                       int64
CpuTurboClock                int64
GpuName                     object
GpuArchitecture             object
GpuBandwidth                 int64
GpuBaseClock                 int64
GpuBoostClock                int64
GpuBus nterface             object
GpuDieSize                 float64
GpuDirectX                  object
GpuFP32Performance           int64
GpuMemoryBus                 int64
GpuMemorySize                int64
GpuMemoryType               object
GpuOpenCL                  float64
GpuOpenGL                  float64
GpuPixelRate                 int64
GpuProcessSize      

In [19]:
# deal with the ordinal column GameSetting
df['GameSetting']=OrdinalEncoder().fit_transform(df[['GameSetting']])

In [20]:
# select all categorical features
categorical_features=["CpuName","GpuName","GpuArchitecture","GpuBus nterface","GpuDirectX","GpuMemoryType","GpuVulkan","GameName","GameResolution","Dataset"]

# function that encodes a feature using panda's get_dummies, appends the new features to the original dataframe and remove the original column
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

# use the function above to encode all the categorical features
for feature in categorical_features:
    df = encode_and_bind(df, feature)

In [21]:
# deal with a column containing " " and "'" in its name
df = df.rename(columns = {"GpuBus nterface_'MXM-B (3.0)'": "GpuBusInterface"})

In [22]:
# we can see that this greatly increases the dimensionality of our data
df.shape

(396028, 605)

### Regression with just part of the features

In [23]:
#shuffle the data
df = shuffle(df)

#use just part of the data
df=df[:1000]
df.shape

(1000, 605)

In [24]:
# rename columns containing "'" 
for i in df.columns:
    old_name=i
    if "'" in i:
        i=i.replace("'", "")
        i=i.replace(" ", "")
        df.columns = df.columns.str.replace(old_name, i)
    #print(i)
df.columns

Index(['CpuNumberOfCores', 'CpuNumberOfThreads', 'CpuBaseClock', 'CpuCacheL1',
       'CpuCacheL2', 'CpuCacheL3', 'CpuFrequency', 'CpuMultiplier',
       'CpuMultiplierUnlocked', 'CpuProcessSize',
       ...
       'GameName_seaOfThieves', 'GameName_starcraft2',
       'GameName_totalWar3Kingdoms', 'GameName_warframe',
       'GameName_worldOfTanks', 'GameResolution_720', 'GameResolution_1080',
       'GameResolution_1440', 'Dataset_fpsbenchmark', 'Dataset_userbenchmark'],
      dtype='object', length=605)

In [25]:
#extract the target variable
y = np.array(df.pop('FPS'))

In [26]:
# scale the data
scaler = StandardScaler()
x_scaled=scaler.fit_transform(df)

/Users/andreibrasoveanu/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/andreibrasoveanu/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [27]:
#split the data for training and testing
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.8, random_state=42)

In [28]:
# fit a standard decision tree regressor implemented in scikit-learn
ScikitDecisionTreeModel=DecisionTreeRegressor(max_depth=50, min_samples_split=2)
ScikitDecisionTreeModel.fit(x_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=50, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [29]:
#make predictions on the test set
y_pred=ScikitDecisionTreeModel.predict(x_test)

In [30]:
# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, y_pred)))

Mean Squared Error: 8876.359662500001
Mean Absolute Error: 60.823375


### Get the most important features

In [31]:
features_importance=ScikitDecisionTreeModel.feature_importances_
indexes=[]
for i in range(len(features_importance)):
    # select only the features that have any relevance
    if features_importance[i]!=0:
        indexes.append(i)
relevant_features=[]
for i in range(len(df.columns)):
    if i in indexes:
        relevant_features.append(df.columns[i])
relevant_features

['CpuNumberOfCores',
 'CpuNumberOfThreads',
 'CpuBaseClock',
 'CpuCacheL1',
 'CpuCacheL2',
 'CpuCacheL3',
 'CpuFrequency',
 'CpuMultiplier',
 'CpuMultiplierUnlocked',
 'CpuProcessSize',
 'CpuTDP',
 'CpuTurboClock',
 'GpuBandwidth',
 'GpuBaseClock',
 'GpuBoostClock',
 'GpuDieSize',
 'GpuFP32Performance',
 'GpuMemoryBus',
 'GpuMemorySize',
 'GpuPixelRate',
 'GpuProcessSize',
 'GpuShaderModel',
 'GpuNumberOfShadingUnits',
 'GpuNumberOfTMUs',
 'GpuTextureRate',
 'GpuNumberOfTransistors',
 'GameSetting',
 'CpuName_AMDFX-4300',
 'CpuName_AMDFX-8370',
 'CpuName_AMDRyzen51500X',
 'CpuName_IntelCorei3-6100',
 'CpuName_IntelCorei5-4440',
 'CpuName_IntelCorei5-650',
 'CpuName_IntelCorei5-6600K',
 'CpuName_IntelCorei5-7500',
 'CpuName_IntelCorei5-8600K',
 'CpuName_IntelCorei7-3770',
 'CpuName_IntelCorei7-3770K',
 'CpuName_IntelCorei7-4770',
 'CpuName_IntelCorei7-6700K',
 'CpuName_IntelCorei7-7700',
 'CpuName_IntelCorei7-8700K',
 'CpuName_IntelCorei9-9900KF',
 'GpuName_NVIDIAGeForceGTX10503GB',
 'G

### Fit another tree with just the most important features

In [34]:
# scale the data
scaler = StandardScaler()
x_scaled=scaler.fit_transform(df[relevant_features])

#split the data for training and testing
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.8, random_state=42)

# fit a new standard decision tree regressor implemented in scikit-learn
ScikitDecisionTreeModel=DecisionTreeRegressor(max_depth=50, min_samples_split=2)
ScikitDecisionTreeModel.fit(x_train, y_train)

#make predictions on the test set
y_pred=ScikitDecisionTreeModel.predict(x_test)

# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(y_test, y_pred)))
print("Mean Absolute Error: " + str(mean_absolute_error(y_test, y_pred)))

Mean Squared Error: 4891.1867125
Mean Absolute Error: 47.99912500000001


### Cross-Validation on scikit's tree implementation

In [36]:
ScikitDecisionTreeModel=DecisionTreeRegressor(max_depth=50, min_samples_split=2)
scores_mse = cross_val_score(ScikitDecisionTreeModel, x_scaled, y, scoring='neg_mean_squared_error', cv=5)
scores_mae = cross_val_score(ScikitDecisionTreeModel, x_scaled, y, scoring='neg_mean_absolute_error', cv=5)

print("Mean Squared Error: " + str(sum(scores_mse)*(-1)/len(scores_mse)))
print("Mean Absolute Error: " + str(sum(scores_mae)*(-1)/len(scores_mae)))

Mean Squared Error: 6695.526052222222
Mean Absolute Error: 52.2925


### Custom Regression Tree Implementation

In [37]:
class CustomDecisionTreeRegressor():
    """
    Class to build a regression decision tree
    """
    def __init__(
        self, 
        Y: list,
        X: pd.DataFrame,
        min_samples_split=None,
        max_depth=None,
        depth=None,
        node_type=None,
        rule=None
    ):
        # save the data to the node 
        self.Y = Y 
        self.X = X

        # save the hyper parameters, define default values
        self.min_samples_split = min_samples_split if min_samples_split else 2
        self.max_depth = max_depth if max_depth else 10

        # default current depth of node 
        self.depth = depth if depth else 0

        # extract all the features
        self.features = list(self.X.columns)

        # type of node 
        self.node_type = node_type if node_type else 'root'

        # rule for spliting 
        self.rule = rule if rule else ""

        # get the mean of Y 
        self.ymean = np.mean(Y)

        # get the residuals 
        self.residuals = self.Y - self.ymean

        # calculate the mse of the node 
        self.mse = self.get_mse(Y, self.ymean)

        # save the number of observations in the node 
        self.n = len(Y)

        # initiate the left and right nodes as empty nodes
        self.left = None 
        self.right = None 

        # default values for splits
        self.best_feature = None 
        self.best_value = None 

    @staticmethod
    def get_mse(ytrue, yhat) -> float:
        """
        method to calculate the mean squared error 
        """
        # get the total number of samples
        n = len(ytrue)

        # get the residuals 
        r = ytrue - yhat 

        # square the residuals 
        r = r ** 2

        # sum the squared residuals
        r = np.sum(r)

        # return the average 
        return r / n

    @staticmethod
    def ma(x: np.array, window: int) -> np.array:
        """
        calculate the moving average of the given list. 
        """
        return np.convolve(x, np.ones(window), 'valid') / window

    def best_split(self) -> tuple:
        """
        given the X features and Y targets calculates the best split 
        for a decision tree
        """
        # create a dataset for spliting
        df = self.X.copy()
        df['Y'] = self.Y

        # get the GINI impurity for the base input 
        mse_base = self.mse

        # find which split yields the best GINI gain 
        #max_gain = 0

        # default best feature and split
        best_feature = None
        best_value = None

        for feature in self.features:
            # drop missing values
            Xdf = df.dropna().sort_values(feature)

            # sort the values and get the rolling average
            xmeans = self.ma(Xdf[feature].unique(), 2)

            for value in xmeans:
                # get the left and right ys 
                left_y = Xdf[Xdf[feature]<value]['Y'].values
                right_y = Xdf[Xdf[feature]>=value]['Y'].values

                # get the means 
                left_mean = np.mean(left_y)
                right_mean = np.mean(right_y)

                # get the left and right residuals 
                res_left = left_y - left_mean 
                res_right = right_y - right_mean

                # concatenate the residuals 
                r = np.concatenate((res_left, res_right), axis=None)

                # calculate the mse 
                n = len(r)
                r = r ** 2
                r = np.sum(r)
                mse_split = r / n

                # check if this is the best split so far 
                if mse_split < mse_base:
                    best_feature = feature
                    best_value = value 

                    # get the best gain to the current one 
                    mse_base = mse_split

        return (best_feature, best_value)

    def fit(self):
        """
        recursive method to create the decision tree
        """
        # make a df from the data 
        df = self.X.copy()
        df['Y'] = self.Y

        # if there is GINI to be gained, we split further 
        if (self.depth < self.max_depth) and (self.n >= self.min_samples_split):

            # get the best split 
            best_feature, best_value = self.best_split()

            if best_feature is not None:
                # save the best split to the current node 
                self.best_feature = best_feature
                self.best_value = best_value

                # gett the left and right nodes
                left_df, right_df = df[df[best_feature]<=best_value].copy(), df[df[best_feature]>best_value].copy()

                # create the left and right nodes
                left = CustomDecisionTreeRegressor(
                    left_df['Y'].values.tolist(), 
                    left_df[self.features], 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split, 
                    node_type='left_node',
                    rule=f"{best_feature} <= {round(best_value, 3)}"
                    )

                self.left = left 
                self.left.fit()

                right = CustomDecisionTreeRegressor(
                    right_df['Y'].values.tolist(), 
                    right_df[self.features], 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split,
                    node_type='right_node',
                    rule=f"{best_feature} > {round(best_value, 3)}"
                    )

                self.right = right
                self.right.fit()

    def print_info(self, width=4):
        """
        print information about one node
        """
        # define the number of spaces 
        const = int(self.depth * width ** 1.5)
        spaces = "-" * const
        
        if self.node_type == 'root':
            print("Root")
        else:
            print(f"|{spaces} Split rule: {self.rule}")
        print(f"{' ' * const}   | MSE of the node: {round(self.mse, 2)}")
        print(f"{' ' * const}   | Count of observations in node: {self.n}")
        print(f"{' ' * const}   | Prediction of node: {round(self.ymean, 3)}")   

    def print_tree(self):
        """
        print the entire tree
        """
        self.print_info() 
        
        if self.left is not None: 
            self.left.print_tree()
        
        if self.right is not None:
            self.right.print_tree()
    def predict(self, dataframe):
        """
        make predictions on a dataframe
        """
        predictions=[]
        for index, row in dataframe.iterrows():
            node=root
            while(node.left is not None and node.right is not None):

                # get the rules for that node
                left_rule=node.left.rule.split(" ")
                right_rule=node.right.rule.split(" ")

                # continue on the left node
                if (left_rule[1]=="<="):
                    if (row[left_rule[0]]<=float(left_rule[2])):
                        node=node.left
                elif (left_rule[1]=="<"):
                    if (row[left_rule[0]]<float(left_rule[2])):
                        node=node.left
                elif (left_rule[1]==">"):
                    if (row[left_rule[0]]>float(left_rule[2])):
                        node=node.left
                else:
                    if (row[left_rule[0]]>=float(left_rule[2])):
                        node=node.left

                # contnue on the right node       
                if (right_rule[1]=="<="):
                    if (row[right_rule[0]]<=float(right_rule[2])):
                        node=node.right
                elif (right_rule[1]=="<"):
                    if (row[right_rule[0]]<float(right_rule[2])):
                        node=node.right
                elif (right_rule[1]==">"):
                    if (row[right_rule[0]]>float(right_rule[2])):
                        node=node.right
                else:
                    if (row[right_rule[0]]>=float(right_rule[2])):
                        node=node.right

                if (node.left is None and node.right is None):
                    predictions.append(node.ymean)

        return predictions

In [39]:
# get only the relevant features
df=df[relevant_features]
df.shape

(1000, 72)

In [40]:
# define the threshold and split the data for holdout
threshold=int(0.8*df.shape[0])
X_train=df[:threshold]
Y_train=y[:threshold]
X_test=df[threshold:]
Y_test=y[threshold:]

In [42]:
# define the tree
root = CustomDecisionTreeRegressor(Y_train, X_train, max_depth=50, min_samples_split=2)
# fit the tree
root.fit()

C:\Users\Mihai\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Mihai\Anaconda3\envs\tensorflow_env\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [43]:
# make predictions
predictions=root.predict(X_test)

In [44]:
# model evaluation using Root Mean Squared Error and Mean Absolute Error
print("Mean Squared Error: " + str(mean_squared_error(Y_test, predictions)))
print("Mean Absolute Error: " + str(mean_absolute_error(Y_test, predictions)))

Mean Squared Error: 4568.418361111111
Mean Absolute Error: 47.721833333333336


### Cross-Validation on our Custom Tree

In [32]:
kf = KFold(n_splits=3, shuffle=False)
#define lists to store te results for each fold
maes=[]
mses=[]

i = 1
for train_index, test_index in kf.split(df):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
        
    # define the tree
    root = CustomDecisionTreeRegressor(y_train, X_train, max_depth=50, min_samples_split=2)
    # fit the tree
    root.fit()
    mse=mean_squared_error(y_test, root.predict(X_test))
    mses.append(mse)
    mae=mean_absolute_error(y_test, root.predict(X_test))
    maes.append(mae)
    
    print(f"Mean Square Error for the fold no. {i} on the test set: {mse}")
    print(f"Mean Absolute Error for the fold no. {i} on the test set: {mae}")

    i += 1

NameError: name 'CustomDecisionTreeRegressor' is not defined

In [50]:
print("Average Mean Squared Error: " + str(sum(mses)/len(mses)))
print("Average Mean Absolute Error: " + str(sum(maes)/len(maes)))

Average Mean Squared Error: 5842.454837525483
Average Mean Absolute Error: 50.674665483847114


In [44]:
class linear_regr:

    def __init__(self):
        from sklearn.linear_model import LinearRegression
        self.model = LinearRegression()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

    def loss(self, X, y, y_pred):
        return mean_squared_error(y, y_pred)

In [ ]:
kf = KFold(n_splits=3, shuffle=False)
#define lists to store te results for each fold
maes=[]
mses=[]

i = 1
for train_index, test_index in kf.split(df):
    X_train = np.array(df.iloc[train_index])
    X_test = np.array(df.iloc[test_index])
    y_train = np.array(y[train_index])
    y_test = np.array(y[test_index])
        
    # define the tree
    model = linear_regr()
    root = ModelTree(model, max_depth=50, min_samples_leaf=2, search_type="greedy")
    # fit the tree
    root.fit(X_train, y_train)
    mse=mean_squared_error(y_test, root.predict(X_test))
    mses.append(mse)
    mae=mean_absolute_error(y_test, root.predict(X_test))
    maes.append(mae)
    
    print(f"Mean Square Error for the fold no. {i} on the test set: {mse}")
    print(f"Mean Absolute Error for the fold no. {i} on the test set: {mae}")

    i += 1